In [0]:
CREATE OR REPLACE TEMP VIEW exploded_genres AS
SELECT 
  movie_id,  
  trim(genre_element) AS genre
FROM hive_metastore.default.dim_movie
LATERAL VIEW explode(split(genre, ",")) AS genre_element;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.dim_genre AS
SELECT 
  ROW_NUMBER() OVER (ORDER BY genre) AS genre_id,  
  TRIM(genre) AS genre
FROM (
  SELECT DISTINCT TRIM(genre) AS genre
  FROM exploded_genres
) t;

In [0]:
SELECT * FROM hive_metastore.default.dim_genre;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.bridge_movie_genre AS
SELECT DISTINCT 
  eg.movie_id, 
  dg.genre_id
FROM exploded_genres eg
JOIN hive_metastore.default.dim_genre dg 
  ON TRIM(eg.genre) = dg.genre;

In [0]:
SELECT 
  bmg.movie_id,  
  COLLECT_LIST(dg.genre) AS genres
FROM hive_metastore.default.bridge_movie_genre bmg
JOIN hive_metastore.default.dim_genre dg  
  ON bmg.genre_id = dg.genre_id
GROUP BY bmg.movie_id
ORDER BY bmg.movie_id;